In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
df = pd.read_csv('./Data_final/CoinText_final.csv')

In [7]:
# 載入模型和tokenizer
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
CionScore=[]
# 設定要分類的標籤
labels = ['positive', 'negative', 'neutral']

for Ctext in df['CoinText']:

    # 將文本轉換為tokens並加入特殊tokens
    inputs = tokenizer(Ctext, return_tensors='pt', padding=True, truncation=True)

    # 進行預測
    outputs = model(**inputs)
    predicted_scores = torch.softmax(outputs.logits, dim=1)

    # 挑選預測中得分最高的情緒類別
    max_score_idx = torch.argmax(predicted_scores)
    max_score_label = labels[max_score_idx]
    max_score = predicted_scores[0][max_score_idx]

    if max_score_label=='positive':
        CionScore.append(max_score)
    elif max_score_label=='negative':
        CionScore.append(-1*max_score)
    else:
        CionScore.append(0)

lo={'CoinDate':df['CoinDate'],'CionScore':CionScore}
df = pd.DataFrame(lo)
df.to_csv('CionScore_final.csv')



# 輸出結果
# print(f"The predicted sentiment for '{text}' is '{max_score_label}' with a score of {max_score:.4f}.")
